### Imports

In [45]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [46]:
# Loading data
distanceDF = pd.read_csv("data/Distance.csv", header=None)
capacityDF = pd.read_csv("data/Capacity.csv", header=None)
populationDF = pd.read_csv("data/Population.csv", header=None)

t_B = 0
t_E = 5
psi = 1
numCounties = 67


# Define sets, parameters, and initial values (example values provided)
I = range(1, numCounties + 1)         # Set of counties
J = range(1, numCounties + 1)         # Set of healthcare facilities (assume same as counties for simplicity)
T = range(t_B, t_E + 1)     # Time periods
T_prime = range(t_B, t_E + 1, psi)  # Decision-making time periods

initial_S = [0.99 for _ in range(numCounties + 1)]
initial_I = [0.01 for _ in range(numCounties + 1)]
initial_R = [0.00 for _ in range(numCounties + 1)]
initial_X = [0.00 for _ in range(numCounties + 1)]


# Parameters (Example values - replace with actual data)
beta = {i: 0.1 for i in I}           # Transmission rate
v = {i: 0.05 for i in I}             # Recovery rate
z = {i: 0.1 for i in I}              # Vaccination rate
l = {i: 0.05 for i in I}             # Leaky vaccine probability
rho = {i: 0.1 for i in I}            # Hospitalization rate
C = {j: capacityDF.iloc[j-1, 0] for j in J}  # Capacity
V_total = {t: 1000 for t in T_prime} # Total vaccines available at each decision period
d = {(i, j): distanceDF.iloc[i-1, j-1] for i in I for j in J}
D_max = 75                           # Max travel distance
N = {i: populationDF.iloc[i-1, 0] for i in I} # Total population in each county
M = 1e6                              # Large number for big-M constraints

# Create a Gurobi model
model = gp.Model("SIRV_Model")

# Decision variables
S = model.addVars(I, T, lb=0, name="S")
I_var = model.addVars(I, T, lb=0, name="I")
R = model.addVars(I, T, lb=0, name="R")
X = model.addVars(I, T, lb=0, name="X")
H = model.addVars(I, T, lb=0, name="H")
V = model.addVars(I, T_prime, lb=0, name="V")
y = model.addVars(I, J, T_prime, lb=0, name="y")
u = model.addVars(I, T_prime, lb=0, name="u")
b = model.addVars(I, J, T_prime, vtype=GRB.BINARY, name="b")
W_SI = model.addVars(I, T, lb=0, name="W_SI") # accounts for W_SI >= 0
Z_XI = model.addVars(I, T, lb=0, name="Z_XI") # accounts for Z_SI >= 0

# Objective: Minimize the total unmet healthcare demand across counties and time
model.setObjective(gp.quicksum(u[i, t] for i in I for t in T_prime), GRB.MINIMIZE)

# Constraints

# SIRV Dynamics
for i in I:
    for t in T:
        if t == t_B:
            # Set initial conditions for S, I, R, X
            model.addConstr(S[i, t] == initial_S[i], name=f"initial_S_{i}_{t}")
            model.addConstr(I_var[i, t] == initial_I[i], name=f"initial_I_{i}_{t}")
            model.addConstr(R[i, t] == initial_R[i], name=f"initial_R_{i}_{t}")
            model.addConstr(X[i, t] == initial_X[i], name=f"initial_X_{i}_{t}")
        else:
            # Update Susceptible, Infected, Recovered, and Vaccinated populations
            model.addConstr(S[i, t] == S[i, t-1] - beta[i] * W_SI[i, t-1] - z[i] * S[i, t-1])
            model.addConstr(I_var[i, t] == I_var[i, t-1] + beta[i] * (W_SI[i, t-1] + l[i] * Z_XI[i, t-1]) - v[i] * I_var[i, t-1])
            model.addConstr(R[i, t] == R[i, t-1] + v[i] * I_var[i, t-1])
            model.addConstr(X[i, t] == X[i, t-1] + z[i] * S[i, t-1] - l[i] * beta[i] * Z_XI[i, t-1])

        # Hospitalization requirement
        model.addConstr(H[i, t] == rho[i] * I_var[i, t])

# Unmet demand for healthcare
for i in I:
    for t in T_prime:
        model.addConstr(u[i, t] == H[i, t] - gp.quicksum(y[i, j, t] for j in J))

# Facility capacity constraint
for j in J:
    for t in T_prime:
        model.addConstr(gp.quicksum(y[i, j, t] for i in I) <= C[j])

# Vaccine allocation constraints
for t in T_prime:
    model.addConstr(gp.quicksum(V[i, t] for i in I) <= V_total[t])

for i in I:
    for t in T_prime:
        model.addConstr(z[i] * S[i, t] <= V[i, t])

# Travel constraints
for i in I:
    for j in J:
        for t in T_prime:
            model.addConstr(y[i, j, t] <= M * b[i, j, t])
            model.addConstr(d[i, j] * b[i, j, t] <= D_max)
            model.addConstr(y[i, j, t] >= b[i, j, t])

# Linearization constraints for auxiliary variables
for i in I:
    for t in T:
        model.addConstr(W_SI[i, t] >= N[i] * I_var[i, t] + S[i, t] * N[i] - N[i] ** 2)
        model.addConstr(W_SI[i, t] <= N[i] * I_var[i, t])
        model.addConstr(W_SI[i, t] <= N[i] * S[i, t])

        model.addConstr(Z_XI[i, t] >= N[i] * I_var[i, t] + X[i, t] * N[i] - N[i] ** 2)
        model.addConstr(Z_XI[i, t] <= N[i] * I_var[i, t])
        model.addConstr(Z_XI[i, t] <= N[i] * X[i, t])

# Optimize model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    for i in I:
        for t in T:
            print(f"Susceptible in county {i} at time {t}: {S[i, t].x}")
            print(f"Infected in county {i} at time {t}: {I_var[i, t].x}")
            print(f"Recovered in county {i} at time {t}: {R[i, t].x}")
            print(f"Vaccinated in county {i} at time {t}: {X[i, t].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 86436 rows, 57486 columns and 201056 nonzeros
Model fingerprint: 0x9822a72c
Variable types: 30552 continuous, 26934 integer (26934 binary)
Coefficient statistics:
  Matrix range     [5e-03, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 4e+06]
Presolve removed 85127 rows and 56313 columns
Presolve time: 0.05s
Presolved: 1309 rows, 1173 columns, 3557 nonzeros
Variable types: 1173 continuous, 0 integer (0 binary)
Found heuristic solution: objective 0.3549769

Explored 1 nodes (0 simplex iterations) in 0.14 seconds (0.07 work units)
Thread count was 22 (of 22 available processors)

Solution count 1: 0.354977 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.549768665